In [1]:
# @title Click ▶ to start
import pandas as pd
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Initialize variables
charges = None
input_widgets = None
uploaded_file = None
# Coulomb's constant
k = 8.99e9  # N m^2 / C^2
# A very small value
epsilon = 1e-10


# Function to create and display the input grid using ipywidgets
def create_input_grid(rows, cols, prefilled_data=None):
    global charges, input_widgets

    # Initialize charges
    charges = np.zeros((rows, cols))

    # If prefilled_data is provided, update charges with it
    if (prefilled_data is not None) and (prefilled_data.shape == (rows, cols)):
        charges = prefilled_data

    # Create a list to hold the FloatText widgets
    input_widgets = []

    # Create FloatText widgets and store them in input_widgets
    for i in range(rows):
        row_widgets = []
        for j in range(cols):
            value = charges[i, j] if prefilled_data is not None else 0.0
            widget = widgets.FloatText(value=value, description=f'[{rows - 1 - i}, {j}]')
            input_widgets.append(widget)
            row_widgets.append(widget)
        display(widgets.HBox(row_widgets))

# Function to handle widget value changes and update charges
def update_charges(change):
    global charges, input_widgets
    rows, cols = charges.shape
    for i in range(rows):
        for j in range(cols):
            charges[i, j] = input_widgets[(rows - 1 - i) * cols + j].value  # Reverse row index

def calculate_density(num_points):
    if num_points >= 5:
        return 0.5  # Base density for 5 points or more
    else:
        base_density = 0.2  # Base density for 1 point
        increment = 0.1     # Increment per additional point
        return base_density + (num_points - 1) * increment

def update_plots(b=None):
    global charges

    if charges is None:
        return  # Return if charges is not initialized

    # Dimensions of the grid
    rows, cols = charges.shape

    # Create a mesh grid for plotting
    X, Y = np.meshgrid(np.arange(cols), np.arange(rows))

    # Initialize electric field components
    Ex = np.zeros((rows, cols))
    Ey = np.zeros((rows, cols))

    # Calculate the electric field components at each grid point
    for y0 in range(rows):
        for x0 in range(cols):
            if charges[y0, x0] != 0:  # Consider only non-zero charges
                for y in range(rows):
                    for x in range(cols):
                        if y == y0 and x == x0:
                            continue  # Skip the charge itself

                        # Calculate the distances in real-world units
                        dx = (x - x0)  # Horizontal distance in meters
                        dy = (y - y0)  # Vertical distance in meters

                        # Compute the distance from charge at (y0, x0) to point (y, x)
                        r = np.sqrt(dx**2 + dy**2)

                        if r > 0:  # Avoid division by zero
                            # Calculate electric field components using Coulomb's law
                            Ex[y, x] += k * charges[y0, x0] * dx / r**3
                            Ey[y, x] += k * charges[y0, x0] * dy / r**3

    # Calculate the magnitude of the electric field
    E_magnitude = np.sqrt(Ex**2 + Ey**2)
    print("Ex: ", Ex)
    print ("E_magnitude: ", E_magnitude)
    # Initialize the color magnitude array with signed values
    E_color_magnitude = np.zeros((rows, cols))

    for y in range(rows):
        for x in range(cols):
            if charges[y, x] != 0:
                # Calculate a signed magnitude based on the charge value
                E_color_magnitude[y, x] = np.sign(charges[y, x]) * E_magnitude[y, x]

    # Add a small epsilon value to avoid division by zero
    E_magnitude += epsilon
    E_color_magnitude += epsilon

    # Normalize electric field components
    Ex_normalized = Ex / E_magnitude
    Ey_normalized = Ey / E_magnitude

    # Clear the plot
    plt.clf()

    # Set up figure and axes
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    fig.suptitle('Electric Field and Equipotential Lines', fontsize=20)

    # First subplot (Quiver Vector Field)
    axs[0].quiver(X, Y, Ex_normalized, Ey_normalized, color='b')
    axs[0].set_title('QUIVER PLOT', fontsize=17)
    axs[0].set_xlabel('x')
    axs[0].set_ylabel('y')
    axs[0].set_xlim(-1, X.max() + 1)
    axs[0].set_ylim(-1, Y.max() + 1)
    axs[0].grid(True)

    # Second subplot (Stream Field)
    stream = axs[1].streamplot(X, Y, Ex_normalized, Ey_normalized,
                               density=calculate_density(np.count_nonzero(charges)), linewidth=1.5,
                               arrowsize=2, color=E_color_magnitude, cmap='cool',
                               integration_direction='both', broken_streamlines=False)

    axs[1].set_title('STREAM PLOT with Equipotential Lines', fontsize=17)
    axs[1].set_xlabel('x')
    axs[1].set_ylabel('y')
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

# Function to handle the file upload and process the first table in the first sheet
def on_file_upload(change):
    global charges, uploaded_file

    # Reset uploaded_file to None to clear previous state
    uploaded_file = None

    # Read the uploaded file into a pandas ExcelFile object
    uploaded_file = change['new']
    file_contents = uploaded_file[list(uploaded_file.keys())[0]]['content']
    xls = pd.ExcelFile(file_contents)

    # Load the first sheet
    sheet_name = xls.sheet_names[0]
    df = pd.read_excel(xls, sheet_name=sheet_name, header=None)

    # Remove non-numeric data and drop rows and columns that are completely NaN
    df = df.apply(pd.to_numeric, errors='coerce').dropna(how='all').dropna(axis=1, how='all')

    # Find the largest contiguous block of numbers
    non_empty_indices = np.argwhere(~np.isnan(df.to_numpy()))
    start_row, start_col = non_empty_indices.min(axis=0)
    end_row, end_col = non_empty_indices.max(axis=0)
    charges = df.iloc[start_row:end_row+1, start_col:end_col+1].to_numpy()

    # Clear the output and display the grid with the new data
    clear_output()
    display(widget_row1, widget_row2)
    create_input_grid(charges.shape[0], charges.shape[1], prefilled_data=charges)
    display(submit_button)

    # Reset the FileUpload widget to clear previous upload state
    excel_upload_button.value.clear()
    excel_upload_button._counter = 0  # Reset the counter that shows the number of uploaded files to 0

# Create widgets for user input
info_heading = widgets.HTML(value="<h1>Electric Field Plotter</h1>")
info_body = widgets.HTML(value="""<div style="line-height: 1.2;">
<h3 style="margin-bottom: 5px;">This tool allows you to visualize electric fields based on a grid of charges.</h3>
<h3><ul style="margin-top: 0px; margin-bottom: 10px;">
<li>Upload an Excel file with the charge values</li>
<li>Or create a table and input the values</li>
<li>Click 'Submit' to update the electric field plots</li>
</ul></h3>
</div>""")

# Define the buttons/widgets to use
table_button = widgets.Button(description="Create Table")
excel_upload_button = widgets.FileUpload(accept='.xlsx', multiple=False, description="Excel Upload")
submit_button = widgets.Button(description="Submit")
rows_widget = widgets.IntSlider(value=5, min=3, max=15, step=1, description='Rows:')
cols_widget = widgets.IntSlider(value=5, min=3, max=15, step=1, description='Columns:')

# Display initial widgets using a VBox for the first row
widget_row1 = widgets.VBox([info_heading, info_body, widgets.HBox([table_button, rows_widget, cols_widget])])
widget_row2 = widgets.HBox([excel_upload_button])

# Function to handle generate button click event
def on_generate_table(b):
    clear_output()
    display(widget_row1, widget_row2)
    create_input_grid(rows_widget.value, cols_widget.value)
    display(submit_button)

# Function to handle submit button click event
def on_submit_button(b):
    update_charges(None)
    update_plots()

# Attach button click events
table_button.on_click(on_generate_table)
submit_button.on_click(on_submit_button)
excel_upload_button.observe(on_file_upload, names='value')

# Initial display of widgets
display(widget_row1, widget_row2)

## Coulomb's Law and Electric Field Calculation

In this section, we describe how the electric field (\(\mathbf{E}\)) is calculated at each
grid point due to a set of point charges.

### Coulomb's Law

Coulomb's Law gives the magnitude of the electric field (\(\mathbf{E}\)) created by a
point charge \(q\) at a distance \(r\) from the charge:

\[
\mathbf{E} = \frac{k_e \cdot q}{r^2} \hat{\mathbf{r}}
\]

where:
- \(k_e\) is Coulomb's constant (\(8.9875 \times 10^9 \, \text{N m}^2 \, \text{C}^{-2}\)),
- \(q\) is the charge,
- \(r\) is the distance from the charge to the point where the electric field is being calculated,
- \(\hat{\mathbf{r}}\) is the unit vector pointing from the charge to the point.

### Electric Field Components

For a charge located at a point \((x_0, y_0)\), the electric field components \(E_x\) and
\(E_y\) at a point \((x, y)\) can be computed as:

\[
E_x = \frac{k_e \cdot q \cdot (x - x_0)}{r^3}
\]
\[
E_y = \frac{k_e \cdot q \cdot (y - y_0)}{r^3}
\]

where:
- \(x - x_0\) is the distance in the \(x\)-direction,
- \(y - y_0\) is the distance in the \(y\)-direction,
- \(r = \sqrt{(x - x_0)^2 + (y - y_0)^2}\) is the distance between the charge and the point.

### Applying Coulomb's Law in the Grid

In our grid, we calculate the electric field at each point due to all other
charges. This involves iterating over all charges and summing their contributions
to the electric field at each point.

1. **Grid Spacing**: We define the spacing between grid points in the \(x\)
   and \(y\) directions as \(\Delta x\) and \(\Delta y\), respectively.

2. **Distance Calculations**: For each charge at \((i, j)\), the distance components \(dx\) and \(dy\) to a point \((m, n)\) on the grid are calculated as:
   \[
   dx = (n - j) \times \Delta x
   \]
   \[
   dy = (m - i) \times \Delta y
   \]
   The distance \(r\) between the charge and the point is then:
   \[
   r = \sqrt{dx^2 + dy^2}
   \]

3. **Field Components Calculation**: The electric field components \(E_x\) and \(E_y\) at point \((m, n)\) due to the charge at \((i, j)\) are:
   \[
   E_x = \frac{k_e \cdot q_{ij} \cdot dx}{r^3}
   \]
   \[
   E_y = \frac{k_e \cdot q_{ij} \cdot dy}{r^3}
   \]
   where \(q_{ij}\) is the charge at \((i, j)\).

4. **Summing Contributions**: We sum the contributions from all charges
   to get the total electric field at each point in the grid.

By iterating over all grid points and charges, we calculate the superposition
of electric fields from multiple charges, which is a fundamental concept in electrostatics.
